In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud
from collections import Counter
import numpy as np

In [ ]:
WINDOW_SIZE, K = 3, 100
VOCAB_SIZE, IN_EMBED_SIZE, OUT_ENBED_SIZE = 30000, 100, 100 
BATCH_SIZE = 128

np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)

vocab_file = 'text8'
#prepare vocabulary
def tokenize(text):
    return text.split(' ')

text = None
with open(vocab_file) as fr:
    text = fr.readlines()[0]
    token_list = tokenize(text)
vocab =  Counter(token_list).most_common(VOCAB_SIZE - 1)
idx_to_word = [item[0] for item in vocab]
idx_to_word.append('UNK')
word_to_idx = {item: i for i, item in enumerate(idx_to_word)}
word_counts = [item[1] for item in vocab]
word_counts.append(len(text) - np.sum(word_counts))
frequence = word_counts / np.sum(word_counts)
frequence = frequence ** (3 / 4)
frequence = frequence / np.sum(frequence)

In [ ]:
#data loader
class MyDataset(tud.Dataset):
    def __init__(self, text, idx_to_word, word_to_idx, WINDOW_SIZE, K, device, frequence):
        super(MyDataset, self).__init__()
        self.text = text
        self.idx_to_word = idx_to_word
        self.word_to_idx = word_to_idx
        self.window_size = WINDOW_SIZE
        self.k = K
        self.device = device
        self.frequence = torch.FloatTensor(frequence)
        self.word_encode = torch.LongTensor([self.word_to_idx.get(word, self.word_to_idx['UNK']) \
                                             for word in self.text])
        
        
    def __len__(self):
        return len(self.word_encode)
    
    def __getitem__(self, idx):
        center_word = self.word_encode[idx]
        pos_index = list(range(idx - self.window_size, idx)) + list(range(idx + 1, idx + self.window_size + 1))
        pos_index = [idx % len(self.word_encode) for idx in pos_index]
        pos_words = self.word_encode[pos_index]
        neg_words = torch.multinomial(self.frequence, self.k * self.window_size * 2, replacement = True)
        return center_word, pos_words, neg_words

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = MyDataset(text, idx_to_word, word_to_idx, WINDOW_SIZE, K, device, frequence)  
dataloader = tud.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = True, num_works = 4)  
for i, (center_word, pos_words, neg_words) in enumerate(dataloader):
    print("iter: {}".format(i))
    print(center_word.shape)
    print(pos_words.shape)
    print(neg_words.shape)
    break

In [123]:
#model
class MyModel(nn.Module):
    def __init__(self, VOCAB_SIZE, IN_EMBED_SIZE, OUT_ENBED_SIZE):
        super(MyModel, self).__init__()
        self.in_embedding = nn.Embedding(VOCAB_SIZE, IN_EMBED_SIZE)
        initrange = 0.5 / IN_EMBED_SIZE
        self.in_embedding.weight.data.uniform_(-initrange, initrange)
        self.out_embedding = nn.Embedding(VOCAB_SIZE, OUT_ENBED_SIZE)
        self.out_embedding.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, center_word, pos_words, neg_words):
        center_embeddings = self.in_embedding(center_word)  #[batch-size, embed_size]
        pos_embeddings = self.in_embedding(pos_words)      #[batch-size, window_size * 2, embed_size]
        neg_embeddings = self.out_embedding(neg_words)      #[batch-size, window_size * 2 * K, embed_size]
        center_embeddings = center_embeddings.unsqueeze(2)                     #[batch-size, embed_size, 1]
        
        pos_dot = pos_embeddings
        
        pos_dot = torch.bmm(pos_embeddings, center_embeddings).squeeze(2)  #[batch-size, window_size * 2]
        neg_dot = torch.bmm(neg_embeddings, -center_embeddings).squeeze(2)  #[batch-size, window_size * 2 * K]
#         print("shape of pos_dot: {}".format(pos_dot.shape))
#         print("shape of neg_dot: {}".format(neg_dot.shape))
        
        pos_loss = F.logsigmoid(pos_dot).sum(1)
        neg_loss = F.logsigmoid(neg_dot).sum(1)
        return - (pos_loss + neg_loss)

In [ ]:
#train
model = MyModel(len(idx_to_word), IN_EMBED_SIZE, OUT_ENBED_SIZE)
learning_rate = 4e-4
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(100):
    for iter, (center_word, pos_words, neg_words) in enumerate(dataloader):
        loss = model(center_word, pos_words, neg_words).mean()
        print('epoch: {}, iter: {}, loss: {}'.format(epoch, iter, loss))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

epoch: 0, iter: 0, loss: 420.04736328125
epoch: 0, iter: 1, loss: 420.03009033203125
epoch: 0, iter: 2, loss: 419.9935302734375
epoch: 0, iter: 3, loss: 419.9306945800781
epoch: 0, iter: 4, loss: 419.844970703125
epoch: 0, iter: 5, loss: 419.7273864746094
epoch: 0, iter: 6, loss: 419.57977294921875
epoch: 0, iter: 7, loss: 419.389892578125
epoch: 0, iter: 8, loss: 419.1412658691406
epoch: 0, iter: 9, loss: 418.9300231933594
epoch: 0, iter: 10, loss: 418.5442810058594
epoch: 0, iter: 11, loss: 418.2599182128906
epoch: 0, iter: 12, loss: 417.8762512207031
epoch: 0, iter: 13, loss: 417.38818359375
epoch: 0, iter: 14, loss: 416.8371276855469
epoch: 0, iter: 15, loss: 416.479736328125
epoch: 0, iter: 16, loss: 415.7598571777344
epoch: 0, iter: 17, loss: 415.1873474121094
epoch: 0, iter: 18, loss: 414.5743713378906
epoch: 0, iter: 19, loss: 413.8144836425781
epoch: 0, iter: 20, loss: 412.9191589355469
epoch: 0, iter: 21, loss: 411.9929504394531
epoch: 0, iter: 22, loss: 410.8089904785156
epo

epoch: 0, iter: 184, loss: 37.19917678833008
epoch: 0, iter: 185, loss: 39.419654846191406
epoch: 0, iter: 186, loss: 39.04886245727539
epoch: 0, iter: 187, loss: 37.275794982910156
epoch: 0, iter: 188, loss: 39.49872589111328
epoch: 0, iter: 189, loss: 36.0349235534668
epoch: 0, iter: 190, loss: 35.45886993408203
epoch: 0, iter: 191, loss: 35.364933013916016
epoch: 0, iter: 192, loss: 35.85647964477539
epoch: 0, iter: 193, loss: 34.69135284423828
epoch: 0, iter: 194, loss: 35.180606842041016
epoch: 0, iter: 195, loss: 37.652652740478516
epoch: 0, iter: 196, loss: 32.84576416015625
epoch: 0, iter: 197, loss: 35.96976852416992
epoch: 0, iter: 198, loss: 37.48291778564453
epoch: 0, iter: 199, loss: 30.677204132080078
epoch: 0, iter: 200, loss: 32.24811553955078
epoch: 0, iter: 201, loss: 27.56901741027832
epoch: 0, iter: 202, loss: 33.04693603515625
epoch: 0, iter: 203, loss: 31.832853317260742
epoch: 0, iter: 204, loss: 36.48822021484375
epoch: 0, iter: 205, loss: 30.751211166381836
epo

epoch: 0, iter: 365, loss: 8.239277839660645
epoch: 0, iter: 366, loss: 6.111988544464111
epoch: 0, iter: 367, loss: 7.9848480224609375
epoch: 0, iter: 368, loss: 7.52569580078125
epoch: 0, iter: 369, loss: 9.409832000732422
epoch: 0, iter: 370, loss: 7.406623363494873
epoch: 0, iter: 371, loss: 6.1284098625183105
epoch: 0, iter: 372, loss: 7.5131425857543945
epoch: 0, iter: 373, loss: 7.837387561798096
epoch: 0, iter: 374, loss: 8.782503128051758
epoch: 0, iter: 375, loss: 9.47379207611084
epoch: 0, iter: 376, loss: 8.982967376708984
epoch: 0, iter: 377, loss: 6.444701671600342
epoch: 0, iter: 378, loss: 6.314486980438232
epoch: 0, iter: 379, loss: 6.62692928314209
epoch: 0, iter: 380, loss: 6.396048545837402
epoch: 0, iter: 381, loss: 6.926065921783447
epoch: 0, iter: 382, loss: 6.265137672424316
epoch: 0, iter: 383, loss: 6.527756214141846
epoch: 0, iter: 384, loss: 6.232541561126709
epoch: 0, iter: 385, loss: 6.864943027496338
epoch: 0, iter: 386, loss: 7.641170024871826
epoch: 0, 

epoch: 0, iter: 547, loss: 3.0254158973693848
epoch: 0, iter: 548, loss: 2.7048699855804443
epoch: 0, iter: 549, loss: 3.013190984725952
epoch: 0, iter: 550, loss: 3.6170153617858887
epoch: 0, iter: 551, loss: 3.7059502601623535
epoch: 0, iter: 552, loss: 3.5170516967773438
epoch: 0, iter: 553, loss: 2.869431495666504
epoch: 0, iter: 554, loss: 2.9924416542053223
epoch: 0, iter: 555, loss: 2.855236053466797
epoch: 0, iter: 556, loss: 3.245734691619873
epoch: 0, iter: 557, loss: 3.768169403076172
epoch: 0, iter: 558, loss: 2.4542808532714844
epoch: 0, iter: 559, loss: 3.2376279830932617
epoch: 0, iter: 560, loss: 2.6136457920074463
epoch: 0, iter: 561, loss: 3.24263596534729
epoch: 0, iter: 562, loss: 3.4135239124298096
epoch: 0, iter: 563, loss: 3.3048596382141113
epoch: 0, iter: 564, loss: 3.497368574142456
epoch: 0, iter: 565, loss: 3.6534507274627686
epoch: 0, iter: 566, loss: 2.747440814971924
epoch: 0, iter: 567, loss: 3.797092914581299
epoch: 0, iter: 568, loss: 4.125815391540527

epoch: 0, iter: 727, loss: 1.5605430603027344
epoch: 0, iter: 728, loss: 1.6026798486709595
epoch: 0, iter: 729, loss: 1.9286208152770996
epoch: 0, iter: 730, loss: 1.6054809093475342
epoch: 0, iter: 731, loss: 2.0922303199768066
epoch: 0, iter: 732, loss: 1.9626744985580444
epoch: 0, iter: 733, loss: 1.4718313217163086
epoch: 0, iter: 734, loss: 1.9255881309509277
epoch: 0, iter: 735, loss: 1.9534367322921753
epoch: 0, iter: 736, loss: 1.731980323791504
epoch: 0, iter: 737, loss: 1.7605023384094238
epoch: 0, iter: 738, loss: 1.793337106704712
epoch: 0, iter: 739, loss: 1.9992640018463135
epoch: 0, iter: 740, loss: 1.8212709426879883
epoch: 0, iter: 741, loss: 1.7735066413879395
epoch: 0, iter: 742, loss: 2.0438787937164307
epoch: 0, iter: 743, loss: 1.6851727962493896
epoch: 0, iter: 744, loss: 1.647029161453247
epoch: 0, iter: 745, loss: 1.4580435752868652
epoch: 0, iter: 746, loss: 1.3663417100906372
epoch: 0, iter: 747, loss: 1.5175318717956543
epoch: 0, iter: 748, loss: 1.33078825

...

epoch: 0, iter: 4011, loss: 0.04145403951406479
epoch: 0, iter: 4012, loss: 0.05950899422168732
epoch: 0, iter: 4013, loss: 0.045301467180252075
epoch: 0, iter: 4014, loss: 0.040846675634384155
epoch: 0, iter: 4015, loss: 0.03344467654824257
epoch: 0, iter: 4016, loss: 0.03446736931800842
epoch: 0, iter: 4017, loss: 0.03308536112308502
epoch: 0, iter: 4018, loss: 0.05282249301671982
epoch: 0, iter: 4019, loss: 0.032847821712493896
epoch: 0, iter: 4020, loss: 0.03490563482046127
epoch: 0, iter: 4021, loss: 0.03663443401455879
epoch: 0, iter: 4022, loss: 0.02999171055853367
epoch: 0, iter: 4023, loss: 0.03177998960018158
epoch: 0, iter: 4024, loss: 0.03334042429924011
epoch: 0, iter: 4025, loss: 0.030656980350613594
epoch: 0, iter: 4026, loss: 0.04254709556698799
epoch: 0, iter: 4027, loss: 0.03957657888531685
epoch: 0, iter: 4028, loss: 0.038009483367204666
epoch: 0, iter: 4029, loss: 0.03632767125964165
epoch: 0, iter: 4030, loss: 0.037944305688142776
epoch: 0, iter: 4031, loss: 0.0332

epoch: 0, iter: 4181, loss: 0.030674487352371216
epoch: 0, iter: 4182, loss: 0.032254815101623535
epoch: 0, iter: 4183, loss: 0.035850122570991516
epoch: 0, iter: 4184, loss: 0.0312664620578289
epoch: 0, iter: 4185, loss: 0.051362182945013046
epoch: 0, iter: 4186, loss: 0.029281210154294968
epoch: 0, iter: 4187, loss: 0.02958858385682106
epoch: 0, iter: 4188, loss: 0.02953571453690529
epoch: 0, iter: 4189, loss: 0.028133120387792587
epoch: 0, iter: 4190, loss: 0.04998622462153435
epoch: 0, iter: 4191, loss: 0.034053679555654526
epoch: 0, iter: 4192, loss: 0.028212204575538635
epoch: 0, iter: 4193, loss: 0.031126342713832855
epoch: 0, iter: 4194, loss: 0.03196826949715614
epoch: 0, iter: 4195, loss: 0.05845057591795921
epoch: 0, iter: 4196, loss: 0.036824166774749756
epoch: 0, iter: 4197, loss: 0.0318756103515625
epoch: 0, iter: 4198, loss: 0.03407017141580582
epoch: 0, iter: 4199, loss: 0.02867729961872101
epoch: 0, iter: 4200, loss: 0.03274432197213173
epoch: 0, iter: 4201, loss: 0.03

epoch: 0, iter: 4351, loss: 0.02927277982234955
epoch: 0, iter: 4352, loss: 0.04101858288049698
epoch: 0, iter: 4353, loss: 0.026149189099669456
epoch: 0, iter: 4354, loss: 0.03058532625436783
epoch: 0, iter: 4355, loss: 0.025981782004237175
epoch: 0, iter: 4356, loss: 0.025127246975898743
epoch: 0, iter: 4357, loss: 0.02841908484697342
epoch: 0, iter: 4358, loss: 0.028140531852841377
epoch: 0, iter: 4359, loss: 0.02877325750887394
epoch: 0, iter: 4360, loss: 0.030990565195679665
epoch: 0, iter: 4361, loss: 0.023151949048042297
epoch: 0, iter: 4362, loss: 0.026628490537405014
epoch: 0, iter: 4363, loss: 0.032148003578186035
epoch: 0, iter: 4364, loss: 0.029642999172210693
epoch: 0, iter: 4365, loss: 0.027427667751908302
epoch: 0, iter: 4366, loss: 0.025624819099903107
epoch: 0, iter: 4367, loss: 0.034097541123628616
epoch: 0, iter: 4368, loss: 0.030527789145708084
epoch: 0, iter: 4369, loss: 0.025506548583507538
epoch: 0, iter: 4370, loss: 0.027124356478452682
epoch: 0, iter: 4371, los

epoch: 0, iter: 4520, loss: 0.022720027714967728
epoch: 0, iter: 4521, loss: 0.035046227276325226
epoch: 0, iter: 4522, loss: 0.03543218597769737
epoch: 0, iter: 4523, loss: 0.023040499538183212
epoch: 0, iter: 4524, loss: 0.02728530392050743
epoch: 0, iter: 4525, loss: 0.023353390395641327
epoch: 0, iter: 4526, loss: 0.02106468938291073
epoch: 0, iter: 4527, loss: 0.02714882418513298
epoch: 0, iter: 4528, loss: 0.027363713830709457
epoch: 0, iter: 4529, loss: 0.023699777200818062
epoch: 0, iter: 4530, loss: 0.029461462050676346
epoch: 0, iter: 4531, loss: 0.027810022234916687
epoch: 0, iter: 4532, loss: 0.023119555786252022
epoch: 0, iter: 4533, loss: 0.028459277004003525
epoch: 0, iter: 4534, loss: 0.025952458381652832
epoch: 0, iter: 4535, loss: 0.024178223684430122
epoch: 0, iter: 4536, loss: 0.027181390672922134
epoch: 0, iter: 4537, loss: 0.023732857778668404
epoch: 0, iter: 4538, loss: 0.03658424690365791
epoch: 0, iter: 4539, loss: 0.02689170092344284
epoch: 0, iter: 4540, loss

epoch: 0, iter: 4689, loss: 0.023363439366221428
epoch: 0, iter: 4690, loss: 0.02483832836151123
epoch: 0, iter: 4691, loss: 0.02563631907105446
epoch: 0, iter: 4692, loss: 0.023706655949354172
epoch: 0, iter: 4693, loss: 0.021381964907050133
epoch: 0, iter: 4694, loss: 0.034299034625291824
epoch: 0, iter: 4695, loss: 0.023337818682193756
epoch: 0, iter: 4696, loss: 0.025489887222647667
epoch: 0, iter: 4697, loss: 0.023217713460326195
epoch: 0, iter: 4698, loss: 0.020878173410892487
epoch: 0, iter: 4699, loss: 0.020464975386857986
epoch: 0, iter: 4700, loss: 0.024138960987329483
epoch: 0, iter: 4701, loss: 0.02485448122024536
epoch: 0, iter: 4702, loss: 0.03061930648982525
epoch: 0, iter: 4703, loss: 0.019500527530908585
epoch: 0, iter: 4704, loss: 0.021148601546883583
epoch: 0, iter: 4705, loss: 0.02011759579181671
epoch: 0, iter: 4706, loss: 0.02045944519340992
epoch: 0, iter: 4707, loss: 0.023330068215727806
epoch: 0, iter: 4708, loss: 0.02150612510740757
epoch: 0, iter: 4709, loss: